In [4]:
# Load packages
import time
from utilities import *

In [9]:
# Count time
time_start = time.time()

ζ_optimal = find_ζ(x_min_RE=1.2,lower=False,bounds=(-1.1,-0.9),ζ_tol=1e-4,ξ_tol=1e-7,max_iter=100,print_option=True)

# Print out the time spent
time_spent = round(time.time()-time_start,4)
print("Time spent: %s seconds ---" % (time_spent))

--- Iteration: 0 ---
Bounds: (-1.1,-0.9)
Error: 0.20000000000000007
--- Iteration: 1 ---
Bounds: (-1.1,-0.9)
Error: 0.20000000000000007
Derivative at mid point: -2.0189654132557466e-05
--- Iteration: 2 ---
Bounds: (-1.1,-0.9999)
Error: 0.10010000000000008
Derivative at mid point: 9.2026198952877e-05
--- Iteration: 3 ---
Bounds: (-1.05005,-0.9999)
Error: 0.05014999999999992
Derivative at mid point: 3.575469491659544e-05
--- Iteration: 4 ---
Bounds: (-1.025075,-0.9999)
Error: 0.025174999999999947
Derivative at mid point: 9.225007598090418e-06
--- Iteration: 5 ---
Bounds: (-1.0125875,-0.9999)
Error: 0.012687499999999963
Derivative at mid point: -1.7199358257058428e-06
--- Iteration: 6 ---
Bounds: (-1.0125875,-1.00614375)
Error: 0.006443750000000081
Derivative at mid point: 2.132240148333686e-06
--- Iteration: 7 ---
Bounds: (-1.009465625,-1.00614375)
Error: 0.0033218750000001407
Derivative at mid point: 1.978074353620185e-06
--- Iteration: 8 ---
Bounds: (-1.0079046875,-1.00614375)
Error: 0

In [10]:
ζ_optimal

-1.007414453125

In [11]:
risk_premia(ζ_optimal,x_min_RE=1.2,lower=False,ξ_tol=1e-7)

0.008142974014493059